In [31]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import glob
import datetime
import itertools

In [10]:
import os
import os.path
import gc

In [3]:
import argparse
parser = argparse.ArgumentParser(description = "Please insert the train flag")

In [4]:
parser.add_argument('-t', '--train', action = "store",
                    help='If true, we train and save. Else, otherwise.', required = True)

_StoreAction(option_strings=['-t', '--train'], dest='train', nargs=None, const=None, default=None, type=None, choices=None, help='If true, we train and save. Else, otherwise.', metavar=None)

In [5]:
my_args = vars(parser.parse_args())
trainFlag = my_args['train']
trainFlag = trainFlag.lower() in ("True", "t", "true", "1", 1)

usage: __main__.py [-h] -t TRAIN
__main__.py: error: argument -t/--train is required


SystemExit: 2

/home/nabeel/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
print datetime.datetime.now()
validFilePaths = []
for f in os.listdir("data/anomaly_data"):
    filePath = os.path.join("data/anomaly_data", f)
    if os.path.isdir(filePath):
        continue
    if os.stat(filePath).st_size <= 3:
        continue
    validFilePaths.append(filePath)
    
numF = int(0.1 * len(validFilePaths))
print 'Using this many files {0}'.format(numF)
validFilePaths = np.random.choice(validFilePaths, numF, replace=False)
df_list = (pandas.read_csv(f) for f in validFilePaths)
df = pandas.concat(df_list, ignore_index=True)
df = df[df['radiant_win'].notnull()]

2017-04-11 20:27:02.710280
Using this many files 999


In [12]:
print df.shape
columns = df.columns
df_catInteger_features_example = filter(lambda x: 'hero_id' in x, columns)

(32191, 331)


In [34]:
from itertools import chain
# these will require string processing on the column names to work
numericalFeatures = ['match_id', 'positive_votes', 'negative_votes', 'first_blood_time', 'radiant_win',
                    'duration', 'kills', 'deaths', 'assists', 'apm', 'kpm', 'kda', 'hero_dmg',
                    'gpm', 'hero_heal', 'xpm', 'totalgold', 'totalxp', 'lasthits', 'denies',
                    'tower_kills', 'courier_kills', 'gold_spent', 'observer_uses', 'sentry_uses',
                    'ancient_kills', 'neutral_kills', 'camps_stacked', 'pings', 'rune_pickups']
categoricalIntegerFeatures = ['barracks_status', 'tower_status', 'hero_id'] 
                              #'item0', 'item1', 'item2', 'item3', 'item4', 'item5']
categoricalFullFeatures = ['patch']
numFeatures = [filter(lambda x: z in x, columns) for z in numericalFeatures]
categoricalIntegerFeatures  = [filter(lambda x: z in x, columns) for z in categoricalIntegerFeatures]
catFull = [filter(lambda x: z in x, columns) for z in categoricalFullFeatures]
numFeatures = list(chain(*numFeatures))
categoricalIntegerFeatures = list(chain(*categoricalIntegerFeatures))
catFull = list(chain(*catFull))

In [58]:
df_numerical = df[numFeatures]
df_numerical.loc[:, 'radiant_win'] = df_numerical.loc[:, 'radiant_win'].apply(lambda x : int(x))
df_cat_num = df[categoricalIntegerFeatures]
df_cat = df[catFull]

#scipy sparse
vectorizer = DictVectorizer(sparse = True)
df_cat = vectorizer.fit_transform(df_cat.fillna('NA').to_dict(orient="records"))

#scipy sparse
# need to make sure that the categorical columns see all fields during training
enc = OneHotEncoder(sparse = True)
fitMatrix = dict.fromkeys(categoricalIntegerFeatures)
heroColumns = [filter(lambda x: z in x, columns) for z in categoricalIntegerFeatures]
barrackColumns = [filter(lambda x: z in x, columns) for z in categoricalIntegerFeatures]
towerColumns = [filter(lambda x: z in x, columns) for z in categoricalIntegerFeatures]
heroesAllCategories = list(set(range(1, 115) + [i[0] for i in \
                            reduce(lambda x, y: x+y, [df_cat_num[i].values for i in heroColumns])]))
barrackAllCategories = list(set([i[0] for i in reduce(lambda x, y: x+y, [df_cat_num[i].values for i in barrackColumns])]))
towerAllCategories = list(set([i[0] for i in reduce(lambda x, y: x+y, [df_cat_num[i].values for i in towerColumns])]))
towerAllCategories =list(itertools.chain.from_iterable(itertools.repeat(towerAllCategories, \
                            1+len(heroesAllCategories)/len(towerAllCategories))))[:len(heroesAllCategories)]
barrackAllCategories=list(itertools.chain.from_iterable(itertools.repeat(barrackAllCategories, \
                            1+len(heroesAllCategories)/len(barrackAllCategories))))[:len(heroesAllCategories)]

for column in heroColumns:
    fitMatrix[column[0]] = heroesAllCategories
for column in towerColumns:
    fitMatrix[column[0]] = towerAllCategories
for column in barrackColumns:
    fitMatrix[column[0]] = barrackAllCategories
enc.fit(pandas.DataFrame.from_dict(fitMatrix))
df_cat_num = enc.transform(df_cat_num)

  (0, 5529)	1.0
  (1, 5602)	1.0
  (3, 7842)	1.0
  (5, 7943)	1.0
  (7, 5603)	1.0
  (9, 7953)	1.0
  (10, 8017)	1.0
  (11, 7953)	1.0
  (12, 5426)	1.0
  (14, 5531)	1.0
  (15, 8017)	1.0
  (16, 7945)	1.0
  (17, 7953)	1.0
  (19, 7831)	1.0
  (20, 7945)	1.0
  (22, 7840)	1.0
  (23, 5515)	1.0
  (24, 7846)	1.0
  (25, 7954)	1.0
  (26, 5603)	1.0
  (27, 4855)	1.0
  (28, 5594)	1.0
  (29, 7831)	1.0
  (30, 7831)	1.0
  (31, 7945)	1.0
  :	:
  (32161, 5539)	1.0
  (32163, 5529)	1.0
  (32164, 5417)	1.0
  (32165, 5428)	1.0
  (32166, 5539)	1.0
  (32167, 7840)	1.0
  (32168, 7943)	1.0
  (32169, 5578)	1.0
  (32171, 7831)	1.0
  (32172, 7954)	1.0
  (32173, 5531)	1.0
  (32174, 5595)	1.0
  (32176, 5515)	1.0
  (32177, 7842)	1.0
  (32178, 5529)	1.0
  (32179, 7953)	1.0
  (32180, 7840)	1.0
  (32181, 5531)	1.0
  (32182, 7954)	1.0
  (32183, 7943)	1.0
  (32185, 7954)	1.0
  (32186, 5428)	1.0
  (32187, 5595)	1.0
  (32188, 7848)	1.0
  (32190, 7953)	1.0


In [ ]:
from scipy.sparse import coo_matrix, hstack

df_cat_num = coo_matrix(df_cat_num)
df_cat = coo_matrix(df_cat)
df = hstack([df_cat_num, df_numerical])

In [ ]:
# df = pandas.concat([df_numerical, df_cat, df_cat_num], ignore_index=True)

In [ ]:
np.random.seed(1)
x = np.random.rand(df.shape[0])
mask = np.where(x < 0.7)[0]
mask1 = np.where(np.logical_and(x >= 0.7, x < 0.9))[0] 
mask2 = np.where(x >= 0.9)[0]

In [ ]:
df_train = df.tocsr()[mask, :]
df_validation = df.tocsr()[mask1, :]
df_test = df.tocsr()[mask2, :]

In [ ]:
NumFeatures = df.shape[1]
layer_size = [10, 10, NumFeatures]
learning_rate = 0.1

In [ ]:
x = tf.placeholder(tf.float32, [None, NumFeatures])
y = x
#encoders
weights_1 = tf.Variable(tf.random_normal([NumFeatures, layer_size[0]]), name='weights_1')
bias_1 = tf.Variable(tf.random_normal([layer_size[0]]), name='bias_1')
weights_2 = tf.Variable(tf.random_normal([layer_size[0], layer_size[1]]), name='weights_2')
bias_2 = tf.Variable(tf.random_normal([layer_size[1]]), name='bias_2')
    
#decoders
weights_3 = tf.Variable(tf.random_normal([layer_size[1], layer_size[2]]), name='weights_3')
bias_3 = tf.Variable(tf.random_normal([layer_size[2]]), name='bias_3')
weights_4 = tf.Variable(tf.random_normal([layer_size[2], NumFeatures]), name='weights_4')
bias_4 = tf.Variable(tf.random_normal([NumFeatures]), name='bias_4')
    
layer1 = tf.nn.relu(tf.matmul(x, weights_1, a_is_sparse=True) + bias_1)
layer2 = tf.nn.relu(tf.matmul(layer1, weights_2, a_is_sparse=True, b_is_sparse=True) + bias_2)
layer3 = tf.nn.relu(tf.matmul(layer2, weights_3, a_is_sparse=True, b_is_sparse=True) + bias_3)
output = tf.nn.relu(tf.matmul(layer3, weights_4, a_is_sparse=True, b_is_sparse=True) + bias_4)
    
cost = tf.reduce_mean(tf.reduce_sum(tf.pow(y-output, 2), 1))
rank = tf.rank(cost)

momentum = 0.5
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum).minimize(cost)
    
variable_dict = {'weights_1': weights_1, 'weights_2': weights_2, 'weights_3': weights_3, 'weights_4': weights_4,
                     'bias_1': bias_1, 'bias_2': bias_2, 'bias_3': bias_3, 'bias_4': bias_4}
saver = tf.train.Saver(variable_dict)
init = tf.global_variables_initializer()

ckpoint_dir = os.path.join(os.getcwd(), 'model-backups/model.ckpt')

In [ ]:
def test(test_data):
    batch = test_data[indices, :].tolil()
    ind = [[[i, batch.rows[i][j]] for j in range(len(batch.rows[i]))] for i in range(batch.shape[0])]
    ind = flatten(ind)
    dat = np.nan_to_num(flatten(batch.data))
    batch = tf.sparse_to_dense(ind, [batch.shape[0], batch.shape[1]], dat)
    data = batch.eval()
    layer1 = tf.nn.relu(tf.matmul(data, weights_1, a_is_sparse=True) + bias_1)
    layer2 = tf.nn.relu(tf.matmul(layer1, weights_2, a_is_sparse=True, b_is_sparse=True) + bias_2)
    layer3 = tf.nn.relu(tf.matmul(layer2, weights_3, a_is_sparse=True, b_is_sparse=True) + bias_3)
    output = tf.nn.relu(tf.matmul(layer3, weights_4, a_is_sparse=True, b_is_sparse=True) + bias_4)

In [ ]:
def train():
    numEpochs = 1000
    numBatches = 100
    batchSize = int(round(0.001 * df_train.shape[0]))
    flatten = lambda l: [item for sublist in l for item in sublist]
    for epochIter in xrange(numEpochs):
        print 'Epoch: {0}'.format(epochIter)
        gc.collect()
        if epochIter % 100 == 0:
            saver.save(sess, ckpoint_dir)
        for batchItr in xrange(numBatches):
            indices = np.random.choice(range(df_train.shape[0]), batchSize, replace=False)
            batch = df_train[indices, :].tolil()
            ind = [[[i, batch.rows[i][j]] for j in range(len(batch.rows[i]))] for i in range(batch.shape[0])]
            ind = flatten(ind)
            dat = np.nan_to_num(flatten(batch.data))
            batch = tf.sparse_to_dense(ind, [batch.shape[0], batch.shape[1]], dat)
            batch = batch.eval()
            sess.run(optimizer, feed_dict = {x : batch})

with tf.Session() as sess:
    if sess.run(rank) != 0:
        raise Exception("Wrong dimenions of cost")
    if (trainFlag):
        sess.run(init)
        train()
    else:
        saver.restore(sess, ckpoint_dir)

In [ ]:
print 'Done'
print datetime.datetime.now()